In [1]:
# ruff: noqa: F401

In [2]:
%load_ext autoreload
%load_ext pyinstrument

%autoreload 2

In [3]:
import sys

sys.path.insert(0, "..")

In [4]:
from datetime import datetime, timedelta
from pathlib import Path

import hvplot.polars
import numpy as np
import polars as pl

In [5]:
from tsdb_benchmarks.monetdb import MonetDB
from tsdb_benchmarks.monetdb.fetch import fetch_binary, fetch_pymonetdb

db = MonetDB()

In [6]:
df = pl.read_parquet("../data/input/data_0.2M_0.5k.parquet")

df

time,col_1,col_2,col_3,col_4,col_5,col_6,col_7,col_8,col_9,col_10,col_11,col_12,col_13,col_14,col_15,col_16,col_17,col_18,col_19,col_20,col_21,col_22,col_23,col_24,col_25,col_26,col_27,col_28,col_29,col_30,col_31,col_32,col_33,col_34,col_35,col_36,…,col_464,col_465,col_466,col_467,col_468,col_469,col_470,col_471,col_472,col_473,col_474,col_475,col_476,col_477,col_478,col_479,col_480,col_481,col_482,col_483,col_484,col_485,col_486,col_487,col_488,col_489,col_490,col_491,col_492,col_493,col_494,col_495,col_496,col_497,col_498,col_499,col_500
datetime[ms],f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,…,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32,f32
2024-08-15 02:41:00,0.524554,0.477075,0.301365,0.086035,0.597285,0.043119,0.133568,0.461486,0.698985,0.373141,0.441279,0.948216,0.824963,0.751381,0.239386,0.646477,0.642296,0.742751,0.889037,0.010748,0.652077,0.617199,0.874318,0.379782,0.882789,0.131134,0.584154,0.460211,0.453917,0.183285,0.792429,0.057061,0.568505,0.415087,0.68145,0.299985,…,0.658785,0.673901,0.911354,0.392003,0.09785,0.930121,0.266884,0.834094,0.338784,0.881258,0.304751,0.555059,0.557787,0.216671,0.915053,0.721279,0.844078,0.208799,0.475463,0.299425,0.27852,0.306467,0.955012,0.889113,0.301954,0.543857,0.257937,0.726755,0.120436,0.639036,0.332407,0.065476,0.736886,0.242331,0.73887,0.265512,0.579258
2024-08-15 02:42:00,0.821767,0.094674,0.734972,0.139922,0.243383,0.933786,0.982846,0.169718,0.175614,0.077493,0.386409,0.89506,0.623241,0.169013,0.298351,0.935542,0.377471,0.837869,0.035134,0.576103,0.266573,0.890418,0.739812,0.682906,0.455877,0.235325,0.022218,0.457534,0.959367,0.854082,0.984065,0.014234,0.619545,0.121988,0.791543,0.035017,…,0.401361,0.602122,0.615821,0.642681,0.643345,0.342692,0.847906,0.711583,0.700303,0.211273,0.13723,0.5056,0.99652,0.997432,0.825113,0.059829,0.244591,0.660734,0.151219,0.108214,0.189322,0.062207,0.233855,0.636825,0.103887,0.54953,0.190117,0.550293,0.738653,0.530214,0.840682,0.481012,0.632614,0.747848,0.946399,0.900591,0.338668
2024-08-15 02:43:00,0.951276,0.73122,0.850451,0.092366,0.531921,0.189276,0.193364,0.592709,0.180091,0.365319,0.17455,0.246438,0.538032,0.701703,0.740081,0.972118,0.904549,0.063419,0.106535,0.989887,0.362526,0.089641,0.379953,0.029863,0.459022,0.37723,0.073348,0.107763,0.977561,0.795237,0.841256,0.751879,0.145434,0.838341,0.958389,0.794217,…,0.598174,0.313713,0.555852,0.160766,0.387838,0.623015,0.782178,0.485091,0.237087,0.807508,0.311178,0.043058,0.598655,0.86598,0.834221,0.354705,0.666874,0.696533,0.627946,0.151129,0.698691,0.177376,0.665253,0.556835,0.96466,0.004346,0.187415,0.63757,0.252799,0.234687,0.217925,0.570648,0.034986,0.586152,0.914273,0.346273,0.621301
2024-08-15 02:44:00,0.709948,0.733406,0.669951,0.011403,0.913939,0.560171,0.639244,0.02121,0.59154,0.981022,0.324198,0.140568,0.478056,0.430004,0.630211,0.181801,0.956042,0.633534,0.85354,0.554621,0.667286,0.397782,0.154186,0.318276,0.747251,0.562591,0.022063,0.027042,0.804215,0.796928,0.551176,0.309716,0.242808,0.719653,0.934386,0.715833,…,0.207523,0.264835,0.460942,0.753063,0.712024,0.718616,0.000127,0.924721,0.655736,0.959097,0.348856,0.638585,0.124401,0.190957,0.94741,0.425781,0.26549,0.919619,0.661019,0.023029,0.141693,0.574829,0.739743,0.033745,0.354654,0.737228,0.492518,0.027382,0.62796,0.693517,0.39653,0.195085,0.568654,0.046906,0.939424,0.791977,0.547603
2024-08-15 02:45:00,0.978002,0.424412,0.632919,0.691689,0.107125,0.248206,0.954185,0.715923,0.931482,0.676695,0.705899,0.046435,0.236004,0.895172,0.458681,0.167613,0.188648,0.769202,0.417068,0.141955,0.940635,0.85125,0.809939,0.928289,0.397367,0.658095,0.970745,0.174064,0.860759,0.465051,0.950083,0.41839,0.22112,0.283718,0.603743,0.668797,…,0.712693,0.976222,0.655139,0.852947,0.543362,0.496692,0.363136,0.236596,0.006429,0.903507,0.48896,0.753444,0.36

In [ ]:
from time import perf_counter

data = {"limit": [], "binary": [], "pymonetdb": []}

for limit in [1, 10, 100, 1_000, 5_000, 10_000, 50_000, 100_000, 500_000, 1_000_000, 5_000_000] + [
    n * 10_000_000 for n in range(1, 5)
]:
    q = f"select * from large order by id limit {limit}"

    t0 = perf_counter()

    fetch_pymonetdb(q, db.connect())

    data["pymonetdb"].append(perf_counter() - t0)

    t0 = perf_counter()

    fetch_binary(q, db.connect())

    data["binary"].append(perf_counter() - t0)

    data["limit"].append(limit)

    print(f"Fetched with limit {limit:_}")

/Users/williamlauren/repos/tsdb-benchmarks/.venv/lib/python3.12/site-packages/sqlalchemy_monetdb/__init__.py:5: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  import pkg_resources
/Users/williamlauren/repos/tsdb-benchmarks/notebooks/../tsdb_benchmarks/monetdb/fetch.py:181: PolarsInefficientMapWarning: 
Series.map_elements is significantly slower than the native series API.
Only use if you absolutely CANNOT implement your logic otherwise.
Replace this expression...
  - s.map_elements(json.loads)
with this one instead:
  + s.str.json_decode()

  return s.map_elements(json.loads, pl.Object)


Fetched with limit 1
Fetched with limit 10
Fetched with limit 100
Fetched with limit 1_000
Fetched with limit 5_000
Fetched with limit 10_000
Fetched with limit 50_000
Fetched with limit 100_000
Fetched with limit 500_000
Fetched with limit 1_000_000
Fetched with limit 5_000_000
Fetched with limit 10_000_000
Fetched with limit 20_000_000


In [8]:
pl.DataFrame(data).hvplot(x="limit", kind="scatter", grid=True, width=900, height=600)

:NdOverlay   [Variable]
   :Scatter   [limit]   (value)